In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import optuna
import sys

# 1. Setup Device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 2. Load Preprocessed Data
try:
    train_df = pd.read_csv('preprocessed_train.csv')
    test_df = pd.read_csv('preprocessed_test.csv')
    original_test = pd.read_csv('test.csv')
except FileNotFoundError:
    print("Error: Preprocessed files not found. Please run the preprocessing step first.")
    sys.exit(1)

# Prepare Data
# Drop target and convert to float32 (standard for PyTorch)
X = train_df.drop('spend_category', axis=1).values.astype(np.float32)
y = train_df['spend_category'].values.astype(np.int64) # Class indices must be Long/Int64
X_submission = test_df.values.astype(np.float32)

# Split for validation during tuning
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Move data to GPU (if dataset fits in VRAM, which this one easily does)
# This speeds up training significantly by avoiding CPU-GPU transfer per batch
X_train_t = torch.tensor(X_train).to(device)
y_train_t = torch.tensor(y_train).to(device)
X_val_t = torch.tensor(X_val).to(device)
y_val_t = torch.tensor(y_val).to(device)
X_submission_t = torch.tensor(X_submission).to(device)

input_dim = X.shape[1]
num_classes = 3 # 0, 1, 2

# 3. Define Optuna Objective
def objective(trial):
    # --- Hyperparameters to tune ---
    n_layers = trial.suggest_int("n_layers", 1, 4)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    
    # Build Model dynamically
    layers = []
    in_features = input_dim
    
    for i in range(n_layers):
        out_features = trial.suggest_int(f"n_units_l{i}", 32, 512)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        in_features = out_features
    
    layers.append(nn.Linear(in_features, num_classes))
    
    model = nn.Sequential(*layers).to(device)
    
    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    # Suggest optimizer type
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop"])
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=lr)

    # --- Training Loop ---
    # We use full-batch training here for speed since dataset is small (12k rows)
    # For larger datasets, use DataLoader with mini-batches
    epochs = 100 
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_t)
        loss = criterion(outputs, y_train_t)
        loss.backward()
        optimizer.step()
        
    # --- Validation ---
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_t)
        _, val_preds = torch.max(val_outputs, 1)
        accuracy = (val_preds == y_val_t).float().mean().item()

    return accuracy

# 4. Run Optimization
print("Starting Optuna optimization...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50) # Run 50 trials

print("\nBest trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# 5. Train Final Model with Best Params
print("\nRetraining best model on full dataset...")

# Reconstruct best model
best_params = study.best_params
layers = []
in_features = input_dim

# We need to rebuild the exact architecture from the best trial
n_layers = best_params["n_layers"]
dropout_rate = best_params["dropout_rate"]

for i in range(n_layers):
    out_features = best_params[f"n_units_l{i}"]
    layers.append(nn.Linear(in_features, out_features))
    layers.append(nn.ReLU())
    layers.append(nn.Dropout(dropout_rate))
    in_features = out_features

layers.append(nn.Linear(in_features, num_classes))
final_model = nn.Sequential(*layers).to(device)

# Optimizer
lr = best_params["lr"]
if best_params["optimizer"] == "Adam":
    optimizer = optim.Adam(final_model.parameters(), lr=lr)
else:
    optimizer = optim.RMSprop(final_model.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss()

# Combine Train + Val for final training
X_full = torch.cat((X_train_t, X_val_t), 0)
y_full = torch.cat((y_train_t, y_val_t), 0)

# Train for a bit longer for final model
final_epochs = 300
for epoch in range(final_epochs):
    final_model.train()
    optimizer.zero_grad()
    outputs = final_model(X_full)
    loss = criterion(outputs, y_full)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 50 == 0:
        print(f"Final Training Epoch {epoch+1}/{final_epochs}, Loss: {loss.item():.4f}")

# 6. Predict on Test Set
final_model.eval()
with torch.no_grad():
    test_outputs = final_model(X_submission_t)
    _, test_preds = torch.max(test_outputs, 1)

# Move back to CPU for pandas
test_preds = test_preds.cpu().numpy()

# Create Submission
submission = pd.DataFrame({
    'trip_id': original_test['trip_id'],
    'spend_category': test_preds
})

submission.to_csv('submission_optuna_gpu.csv', index=False)
print("Submission saved to 'submission_optuna_gpu.csv'")

c:\Users\gupta\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


[I 2025-11-27 18:15:42,242] A new study created in memory with name: no-name-e27ede13-f7b7-418c-a2ec-1bc928466f5a


Starting Optuna optimization...


[I 2025-11-27 18:15:56,826] Trial 0 finished with value: 0.7396988868713379 and parameters: {'n_layers': 3, 'dropout_rate': 0.16229319214747512, 'lr': 0.0068088306815908516, 'n_units_l0': 470, 'n_units_l1': 206, 'n_units_l2': 383, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 0.7396988868713379.
[I 2025-11-27 18:15:58,079] Trial 1 finished with value: 0.7258319854736328 and parameters: {'n_layers': 1, 'dropout_rate': 0.36979728126715694, 'lr': 0.0002697052212158146, 'n_units_l0': 96, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.7396988868713379.
[I 2025-11-27 18:16:11,591] Trial 2 finished with value: 0.7428684830665588 and parameters: {'n_layers': 3, 'dropout_rate': 0.2389371390577397, 'lr': 0.009017944278541414, 'n_units_l0': 235, 'n_units_l1': 410, 'n_units_l2': 310, 'optimizer': 'RMSprop'}. Best is trial 2 with value: 0.7428684830665588.
[I 2025-11-27 18:16:15,818] Trial 3 finished with value: 0.7301901578903198 and parameters: {'n_layers': 1, 'dropout_rate': 0.237068


Best trial:
  Value: 0.7610934972763062
  Params: 
    n_layers: 2
    dropout_rate: 0.49252545505543543
    lr: 0.0008024154579324329
    n_units_l0: 375
    n_units_l1: 359
    optimizer: RMSprop

Retraining best model on full dataset...
Final Training Epoch 50/300, Loss: 0.5666
Final Training Epoch 100/300, Loss: 0.5040
Final Training Epoch 150/300, Loss: 0.4464
Final Training Epoch 200/300, Loss: 0.3983
Final Training Epoch 250/300, Loss: 0.3577
Final Training Epoch 300/300, Loss: 0.3307
Submission saved to 'submission_optuna_gpu.csv'
